<a href="https://colab.research.google.com/github/shaifuss/data_science_seminar/blob/master/topic_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import pandas as pd
import json



In [20]:
def load_reviews():
  # load stuff
  with open(r'/content/drive/My Drive/Data Science Class/pizza_reviews.json', 'r') as f:
    pizza_reviews = json.load(f)
  return pizza_reviews
review_list = load_reviews()

In [21]:
review_df = pd.DataFrame(review_list)

text_df = review_df[['review_id', 'text']].copy()
text_df.head()

,review_id,text
0,mM8i91yWP1QbImEvz5ds0w,"In the heart of Chinatown, I discovered it enr..."
1,09qxjFi4abaW66JeSLazuQ,Was a Chicago style deep dish. Homemade type ...
2,K-wdPGHbErfxbKK6PetrmA,First time eating there and everything was so ...
3,jkVxX4ieJwVRO9n4E8tNMw,More than just Pizza. This location is small ...
4,Lb9r62Qlu12ZB909CbFeOQ,I ordered a pizza at 4:49. Got an email that s...


word level preprocessing utils

In [7]:
pip install pyspellchecker

     |████████████████████████████████| 1.9MB 4.4MB/s 


In [22]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import re

from spellchecker import SpellChecker

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [23]:
def regex_filter(sentence):
    # fix missing delimiter - i.e deepDishPizza
    sentence = re.sub(r'([a-z])([A-Z])', r'\1\. \2', sentence)
    sentence = sentence.lower()
    sentence = re.sub(r'&gt|&lt', ' ', sentence)
    # fix letter repetition (if more than 2)
    sentence = re.sub(r'([a-z])\1{2,}', r'\1', sentence)
    # fix non-word repetition (if more than 1)
    sentence = re.sub(r'([\W+])\1{1,}', r'\1', sentence)
    # string * as delimiter
    sentence = re.sub(r'\*|\W\*|\*\W', '. ', sentence)
    # xxx[?!]. -- > xxx.
    sentence = re.sub(r'\W+?\.', '.', sentence)
    # [.?!] --> [.?!] xxx
    sentence = re.sub(r'(\.|\?|!)(\w)', r'\1 \2', sentence)
    # fix phrase repetition
    sentence = re.sub(r'(.{2,}?)\1{1,}', r'\1', sentence)

    return s.strip()

In [24]:
# remove numbers and punctuation marks
def filter_punctuation(word_list):
    return [word for word in word_list if word.isalpha()]

# remove unimportant connective words such as "and", "the", etc
def filter_stopwords(word_list):
  return [word for word in word_list if word not in stopwords.words('english')]

# TODO is this helpful or not?
def retain_nouns_verbs(word_list):
    return [word for (word, pos) in nltk.pos_tag(word_list) if pos[:2] in ['NN']] #, 'VBP', 'VBN']]

# normlize for part of speech
def stem_words(word_list):
  ps = PorterStemmer()
  return [ps.stem(word) for word in word_list]

def fix_spelling(word_list):
  spell = SpellChecker()
  return [spell.correction(word) for word in word_list]

In [26]:
def pre_proccess_words(text):
  word_list = word_tokenize(text)
  word_list = filter_punctuation(word_list)
  word_list = fix_spelling(word_list) 
  word_list = filter_stopwords(word_list)
  word_list = retain_nouns(word_list)
  return stem_words(word_list)

In [27]:
def pre_process(reviews):
  print('Stage 1: Preprocess raw review texts')
  review_count = len(reviews)
  sentences = []  
  token_lists = []  
  #index_list = []
  for i, idx in enumerate(reviews):
      sentence = regex_filter(reviews[idx])
      tokens = pre_process_words(sentence)
      if tokens:
          #idx_list.append(idx)
          sentences.append(sentence)
          token_lists.append(tokens)
  return sentences, token_lists

In [ ]:
sentences, token_lists, idx_in = pre_process(text_df.text)